In [1]:
from amazonproduct import API
import os
from nltk.corpus.reader.plaintext import CategorizedPlaintextCorpusReader
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
import string
from nltk.classify import NaiveBayesClassifier, util
import random
import pandas as pd
import re

In [1]:
cfg = {
    'access_key': 'ABCDEFG1234X',
    'secret_key': 'Ydjkei78HdkffdklieAHDJWE3134',
    'associate_tag': 'redtoad-10',
    'locale': 'us'
}

In [3]:
api = API(cfg = cfg)

In [ ]:
"""
'All','Wine','Wireless','ArtsAndCrafts','Miscellaneous','Electronics','Jewelry','MobileApps','Photo','Shoes','KindleStore','Automotive','Vehicles','Pantry','MusicalInstruments','DigitalMusic','GiftCards','FashionBaby','FashionGirls','GourmetFood','HomeGarden','MusicTracks','UnboxVideo','FashionWomen','VideoGames','FashionMen','Kitchen','Video','Software','Beauty','Grocery',,'FashionBoys','Industrial','PetSupplies','OfficeProducts','Magazines','Watches','Luggage','OutdoorLiving','Toys','SportingGoods','PCHardware','Movies','Books','Collectibles','Handmade','VHS','MP3Downloads','HomeAndBusinessServices','Fashion','Tools','Baby','Apparel','Marketplace','DVD','Appliances','Music','LawnAndGarden','WirelessAccessories','Blended','HealthPersonalCare','Classical'
"""

In [44]:
"Fashion"
"FashionMen"
"FashionWomen"
shoes_running = {}
#shoes_running['ind'] = "Fashion"
shoes_running['f'] = 679360011
shoes_running['m'] = 679286011

# dict_shoes = {'index':[], 'node'}
"Beauty" 
"HealthPersonalCare"
perfume = {}
#perfume['ind'] = 'Beauty'
perfume['f'] = 11056931
perfume['m'] = 11056761

"Fashion"
watch = {}
watch['f'] = 6358543011
watch['m'] = 6358539011

razor = {}
razor['f'] = 13269991011

"""Cartridges & Refills (13269995011a)
Disposable Razors (13269997011a)
Razor Systems (13269993011a)
"""

razor['m'] = 13271080011

"""Razor Systems (13271082011a)
Cartridges & Refills (13271084011a)
Disposable Razors (13271086011a)"""

"Fasion"
eyewear = {}
eyewear['m'] = 7072330011
eyewear['f'] = 7072321011

glasses = {}
glasses['m'] = 2474995011
glasses['f'] = 2474971011

socks = {}
socks['f'] = 1044886

"""Casual Socks (2376196011a)
Dress & Trouser Socks (2376197011a)
Athletic Socks (1044920a)
Leg Warmers (2376198011a)
Tights (1044936a)
Sheers (1044934a)
Slipper Socks (2376199011a)
No Show & Liner Socks (2376200011a)"""

socks['m'] = 1045708

"""Casual Socks (2476509011a)
Dress & Trouser Socks (1045726a)
Athletic Socks (1045724a)"""

wallets = {}
wallets['m'] = 2475895011
wallets['f'] = 2475898011

"Fashion"
"SportingGoods"
shirts = {}
shirts['m'] = 11444073011
shirts['f'] = 11444120011

"Health"
vitamins = {}
vitamins['m'] = 3760901
vitamins['f'] = 3764441

""
pants = {}
pants['f'] = 11443933011
pants['m'] = 11443885011

In [69]:
Shoes_Corpus = build_corpus("SportingGoods", "ShoesCorpus", shoes_running)

In [71]:
Shoes_Corpus = build_corpus("Fashion", "ShoesCorpus", shoes_running)

In [78]:
ShoesFD = get_FD(Shoes_Corpus)
shoes_features = create_features(Shoes_Corpus)
shoesSC = NB_classify(shoes_features, 0.8)
shoes_info = store_most_informative_features(shoesSC, 30)

[(u'shoe', 583), (u'foot', 569), (u'upper', 535), (u'us', 491), (u'length', 473), (u'shoes', 449), (u'comfort', 442), (u'fit', 429), (u'running', 406), (u'outsole', 397), (u'mesh', 385), (u'cushioning', 378), (u'lightweight', 370), (u'size', 359), (u'5', 354)]
84.4349680171
72.3404255319


In [85]:
graph_features(shoes_info, "Running Shoes")

AttributeError: unexpected attribute 'range_padding' to FactorRange, possible attributes are bounds, callback, factors, js_event_callbacks, js_property_callbacks, max_interval, min_interval, name, offset, subscribed_events or tags

In [45]:
Pants_Corpus = build_corpus("SportingGoods", "PantsCorpus", pants)

In [55]:
Pants_Corpus = build_corpus("Fashion", "PantsCorpus", pants)

In [67]:
PantsFD = get_FD(Pants_Corpus)
pants_features = create_features(Pants_Corpus)
pantsSC = NB_classify(pants_features, 0.8)
pants_info = store_most_informative_features(pantsSC, 30)

[(u'waist', 1061), (u'pants', 970), (u'pant', 782), (u'",', 762), (u'fabric', 639), (u'fit', 589), (u'5', 578), (u'length', 557), (u'size', 537), (u'pockets', 518), (u'keep', 437), (u'"),', 420), (u'snow', 415), (u'dry', 408), (u'waterproof', 400)]
90.1939655172
81.3852813853


In [68]:
graph_features(pants_info, "Pants")

In [56]:
Shirt_Corpus = build_corpus("SportingGoods", "ShirtCorpus", shirts)
Shirt_Corpus = build_corpus("Fashion", "ShirtCorpus", shirts)

In [58]:
ShirtFD = get_FD(Shirt_Corpus)
shirt_features = create_features(Shirt_Corpus)

[(u'fabric', 1044), (u'size', 837), (u'shirt', 680), (u'length', 672), (u'fit', 656), (u'sweat', 641), (u'compression', 611), (u'dry', 544), (u'moisture', 512), (u'odor', 508), (u'anti', 491), (u'body', 482), (u'skin', 458), (u'top', 452), (u'soft', 446)]


In [59]:
shirtSC = NB_classify(shirt_features, 700)
shirtSC.show_most_informative_features(30)

92.1428571429
88.1057268722
Most Informative Features
                     bra = 1                   f : m      =     31.0 : 1.0
               promotoes = 1                   m : f      =     21.0 : 1.0
                heatgear = 1                   m : f      =     19.7 : 1.0
                inspired = 1                   f : m      =     19.0 : 1.0
                monterey = 1                   f : m      =     17.7 : 1.0
                    tail = 1                   f : m      =     16.3 : 1.0
                     men = 1                   m : f      =     15.9 : 1.0
                 pinhole = 1                   f : m      =     15.7 : 1.0
                   swing = 1                   f : m      =     15.0 : 1.0
                   women = 1                   f : m      =     14.7 : 1.0
              safeguards = 1                   m : f      =     13.8 : 1.0
                      +) = 1                   m : f      =     13.8 : 1.0
                 updated = 1                  

In [153]:
Wallet_Corpus = build_corpus("Fashion", "WalletCorpus", wallets)

No child nodes found
No child nodes found


In [154]:
WalletFD = get_FD(Wallet_Corpus)
wallet_features = create_features(Wallet_Corpus)

[(u'p', 598), (u'wallet', 500), (u'cards', 436), (u'rfid', 390), (u'leather', 305), (u'card', 261), (u'1', 212), (u'credit', 194), (u'x', 176), (u'id', 160), (u'blocking', 150), (u'strong', 148), (u'2', 143), (u'></', 143), (u'3', 130)]


In [155]:
walletSC = NB_classify(wallet_features, 0.8)
walletSC.show_most_informative_features(25)

98.0
84.0909090909
Most Informative Features
                 fashion = 1                   f : m      =     10.3 : 1.0
                  zipper = 1                   f : m      =      9.7 : 1.0
                    clip = 1                   m : f      =      7.4 : 1.0
                   women = 1                   f : m      =      7.4 : 1.0
            thanksgiving = 1                   f : m      =      6.3 : 1.0
                 provide = 1                   f : m      =      6.3 : 1.0
                  change = 1                   f : m      =      6.3 : 1.0
                    bags = 1                   f : m      =      6.2 : 1.0
              minimalist = 1                   m : f      =      6.2 : 1.0
                   sleek = 1                   m : f      =      5.7 : 1.0
                zippered = 1                   f : m      =      5.7 : 1.0
              experience = 1                   f : m      =      5.0 : 1.0
                  bifold = 1                   m : f   

In [124]:
Socks_Corpus = build_corpus("Fashion", "SocksCorpus", socks, nodelimit = 2)

In [125]:
Socks_Corpus = build_corpus("SportingGoods", "SocksCorpus", socks, nodelimit = 2)

In [126]:
SocksFD = get_FD(Socks_Corpus)
sock_features = create_features(Socks_Corpus)

[(u'socks', 2674), (u'cotton', 625), (u'comfort', 582), (u'feet', 572), (u'size', 535), (u'sock', 505), (u'quality', 478), (u'fit', 454), (u'comfortable', 426), (u'dry', 422), (u'high', 420), (u'toe', 377), (u'5', 374), (u'women', 369), (u'soft', 363)]


In [127]:
socksSC = NB_classify(sock_features)
socksSC.show_most_informative_features(20)

90.5598243688
81.5789473684
Most Informative Features
                    knee = 1                   f : m      =     17.9 : 1.0
                  argyle = 1                   m : f      =     13.8 : 1.0
                  formal = 1                   m : f      =     13.2 : 1.0
                    cute = 1                   f : m      =     12.1 : 1.0
            manufacturer = 1                   m : f      =     10.9 : 1.0
                     gym = 1                   f : m      =     10.3 : 1.0
                  giving = 1                   f : m      =      9.8 : 1.0
             comfortably = 1                   f : m      =      9.2 : 1.0
                exercise = 1                   f : m      =      8.6 : 1.0
                   range = 1                   f : m      =      8.6 : 1.0
                     men = 1                   m : f      =      8.0 : 1.0
                     six = 1                   f : m      =      8.0 : 1.0
                 similar = 1                  

In [130]:
socks_info = store_most_informative_features(socksSC,30)
graph_features(socks_info, "Socks")

In [98]:
Eyewear_Corpus = build_corpus("Fashion", "EyewearCorpus", eyewear)

In [100]:
EyewearFD = get_FD(Eyewear_Corpus)
eyewear_features = create_features(Eyewear_Corpus)

[(u'p', 1749), (u'lenses', 1581), (u'glasses', 1291), (u'li', 1174), (u'sunglasses', 1049), (u'1', 883), (u'lens', 771), (u'case', 665), (u'polarized', 628), (u'frame', 523), (u'100', 517), (u'quality', 506), (u'protection', 491), (u'strong', 437), (u'></', 420)]


In [106]:
eyewearSC = NB_classify(eyewear_features, 0.8)
eyewearSC.show_most_informative_features(25)

76.625
60.0
Most Informative Features
                    fact = 1                   f : m      =     14.3 : 1.0
                    /></ = 1                   f : m      =     14.3 : 1.0
              industries = 1                   f : m      =     13.7 : 1.0
                 golfers = 1                   f : m      =     13.0 : 1.0
                 tension = 1                   f : m      =     13.0 : 1.0
                     ask = 1                   f : m      =     13.0 : 1.0
                 shipped = 1                   m : f      =     11.0 : 1.0
                      co = 1                   f : m      =      8.6 : 1.0
                   names = 1                   f : m      =      8.6 : 1.0
                      hd = 1                   f : m      =      8.6 : 1.0
                 weekend = 1                   f : m      =      8.2 : 1.0
                    hide = 1                   f : m      =      8.2 : 1.0
                       r = 1                   f : m      =   

In [78]:
Razor_Corpus = build_corpus("HealthPersonalCare", "RazorCorpus", razor)

In [79]:
RazorFD = get_FD(Razor_Corpus)

[(u'razor', 1418), (u'p', 775), (u'blades', 747), (u'blade', 742), (u'shave', 734), (u'skin', 479), (u'shaving', 459), (u'razors', 437), (u'handle', 295), (u'edge', 236), (u'safety', 233), (u'smooth', 227), (u'gillette', 226), (u'5', 217), (u'close', 205)]


In [80]:
razor_features = create_features(Razor_Corpus)

In [105]:
razorSC = NB_classify(razor_features, 0.8)
razorSC.show_most_informative_features(25)

93.345323741
92.8104575163
Most Informative Features
               intuition = 1                   f : m      =     34.0 : 1.0
                straight = 1                   m : f      =     25.3 : 1.0
                    step = 1                   f : m      =     22.3 : 1.0
                  curves = 1                   f : m      =     19.9 : 1.0
            moisturizing = 1                   f : m      =     19.1 : 1.0
                  shower = 1                   f : m      =     17.6 : 1.0
                 leaving = 1                   f : m      =     17.6 : 1.0
                    legs = 1                   f : m      =     15.2 : 1.0
                   women = 1                   f : m      =     14.0 : 1.0
                    edge = 1                   m : f      =     13.6 : 1.0
                    many = 1                   m : f      =     11.7 : 1.0
                     end = 1                   m : f      =     11.7 : 1.0
                  double = 1                   

In [22]:
Watch_Corpus = build_corpus("Fashion", "WatchesCorpus", watch)

In [29]:
WatchFD = get_FD(Watch_Corpus)

[(u'watch', 2145), (u'p', 1513), (u'1', 920), (u'case', 753), (u'steel', 720), (u'stainless', 614), (u'dial', 510), (u'time', 484), (u'band', 460), (u'width', 434), (u'2', 374), (u'q', 342), (u'movement', 340), (u'warranty', 331), (u'features', 327)]


In [30]:
watch_feature = create_features(Watch_Corpus) 

In [115]:
watchSC = NB_classify(watch_feature, 400)
watchSC.show_most_informative_features(25)

80.0
61.9047619048
Most Informative Features
                  womens = 1                   f : m      =     14.3 : 1.0
                 reserve = 1                   m : f      =     11.0 : 1.0
                  highly = 1                   m : f      =      9.0 : 1.0
                   pearl = 1                   f : m      =      9.0 : 1.0
                   money = 1                   f : m      =      9.0 : 1.0
                  ladies = 1                   f : m      =      8.4 : 1.0
                    pink = 1                   f : m      =      8.3 : 1.0
                pressure = 1                   m : f      =      8.3 : 1.0
                  mother = 1                   f : m      =      8.3 : 1.0
                 buttons = 1                   m : f      =      8.3 : 1.0
                  sensor = 1                   m : f      =      8.3 : 1.0
                 compass = 1                   m : f      =      7.7 : 1.0
                  triple = 1                   m : f   

In [80]:
Perfume_Corpus = build_corpus("Beauty", "PerfumeCorpus", perfume)
Perfume_Corpus = build_corpus("HealthPersonalCare", "PerfumeCorpus", perfume)

In [81]:
PerfumeFD = get_FD(Perfume_Corpus)

[(u'fragrance', 1085), (u'notes', 642), (u'skin', 515), (u'scent', 492), (u'spray', 465), (u'body', 454), (u'oz', 431), (u'men', 370), (u'eau', 356), (u'oil', 343), (u'affect', 333), (u'wear', 317), (u'3', 310), (u'blend', 280), (u'recommended', 256)]


In [82]:
perfume_feature = create_features(Perfume_Corpus)

In [88]:
perfumeSC = NB_classify(perfume_feature, 0.8)
perfumeSC.show_most_informative_features(30)

91.2835249042
81.9230769231
Most Informative Features
               essential = 1                   f : m      =     25.1 : 1.0
                 tobacco = 1                   m : f      =     21.2 : 1.0
                rosewood = 1                   m : f      =     18.8 : 1.0
                     man = 1                   m : f      =     16.1 : 1.0
                sycamore = 1                   m : f      =     14.8 : 1.0
                   homme = 1                   m : f      =     13.2 : 1.0
                   women = 1                   f : m      =     11.4 : 1.0
               deodorant = 1                   m : f      =     11.3 : 1.0
                   manly = 1                   m : f      =     10.8 : 1.0
                moisture = 1                   m : f      =     10.8 : 1.0
                    sage = 1                   m : f      =     10.8 : 1.0
                delicate = 1                   f : m      =     10.8 : 1.0
                     men = 1                  

In [101]:
perfume_info = store_most_informative_features(perfumeSC,30)
graph_features(perfume_info, "Perfume")

In [37]:
from bokeh.io import show
from bokeh.models import ColumnDataSource, PrintfTickFormatter, LabelSet
from bokeh.plotting import figure
from bokeh.models.glyphs import HBar
from bokeh.io import output_notebook
from bokeh.models import Range1d, CategoricalAxis, FactorRange
output_notebook()

Loading BokehJS ...

In [84]:
def graph_features(featlist, name):
    """
    Function takes as input, a list of features as outputed by store_most_informative_features, and a str for product
    Function creates a labeled dataframe, transforms the df by
        retaining positive values for "f" features, negative values for "m" features, sorting by descending order
        use Bokeh to graph the most informative features from most "f" indicating to most "m" indicating
    Function returns the DataFrame
    """
    featdf = pd.DataFrame(featlist)
    featdf.columns = ['word','gender','ratio']
    if featdf[featdf.gender=='m'].ratio.all > 0:
        featdf.loc[featdf.gender=='m','ratio'] =-1*featdf[featdf.gender=='m'].ratio
    featdf.sort_values(by='ratio',ascending = True, inplace = True)
    x_max = featdf.ratio.abs().max() + 5
    p = figure(y_range=list(featdf.word), x_range=(-x_max,x_max), 
               title="Most Informative Words in Descriptions for " + name)
    
#    p.extra_y_ranges = {"foo": FactorRange(factors=list(featdf.word))}
#    p.add_layout(CategoricalAxis(y_range_name="foo"), "center")

    p.hbar(y='word', right='ratio', left = 0, height = 0.8, legend = "Female",
           color = 'deeppink', source=ColumnDataSource(featdf[featdf.gender=='f']))
    p.hbar(y='word', left ='ratio', right = 0, height = 0.8, legend = "Male",
           color = 'deepskyblue', source=ColumnDataSource(featdf[featdf.gender=='m']))
    labels_m = LabelSet(x=0, y='word', text='word', level='glyph', x_offset = 5,
              source=ColumnDataSource(featdf[featdf.gender=='m']), y_offset = -9, render_mode='canvas')
    labels_f = LabelSet(x=0, y='word', text='word', level='glyph', x_offset = -5, text_align = 'right',
              source=ColumnDataSource(featdf[featdf.gender=='f']), y_offset = -9, render_mode='canvas')
    p.add_layout(labels_m)
    p.add_layout(labels_f)
    p.yaxis.visible = False
    p.y_range.range_padding = 0.1
    p.xaxis.axis_label = "Likelihood Ratio"
    p.xaxis.axis_label_text_font_size = '1em'
    p.xaxis.major_label_text_font_size = '1em'
    p.title.text_font_size = '1.2em'
    p.xaxis[0].formatter = PrintfTickFormatter(format="%uX")
    p.legend.location = "bottom_right"
    p.legend.label_text_font_size = '1em'
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    show(p)
#    return featdf

In [7]:
def store_most_informative_features(self, n=20):
    # function acts on a Naive Bayes classifier object outputed by NLTK
    # function outputs list of n most informative features, the label of lean, and the ratio
    cpdist = self._feature_probdist
    feat_list = []

    for (fname, fval) in self.most_informative_features(n):
        def labelprob(l):
            return cpdist[l, fname].prob(fval)

        labels = sorted([l for l in self._labels
                         if fval in cpdist[l, fname].samples()],
                        key=labelprob)
        if len(labels) == 1:
            continue
        l0 = labels[0]
        l1 = labels[-1]
        if cpdist[l0, fname].prob(fval) == 0:
            ratio = 'INF'
        else:
            ratio = (cpdist[l1, fname].prob(fval) /
                               cpdist[l0, fname].prob(fval))
        feat_list.append([fname, ("%s" % l1)[:6], ratio])
    return feat_list

In [8]:
def NB_classify(feature, fraction = 0.8, cats = ['f','m']):
    """
    Function takes as inputs:
        feature(dic) as dictionary of labeled features
        fraction(0-1.0) as proportion of raw files to include in the training set
    Function scrambles file order randomly,
        takes the portion of training data to train the NLTK Naive Bayes classifier, prints its accuracy,
        prints accuracy of the classifier on the test data
    Function outputs the Naive Bayes Sentiment classifier object
    """
    test = []
    train = []
    for cat in cats:
        featlist = feature[cat]
        split = int(round(len(featlist) * fraction))
        random.shuffle(featlist)
        train += featlist[:split]
        test += featlist[split:]
    sentiment_classifier = NaiveBayesClassifier.train(train)
    print(util.accuracy(sentiment_classifier, train) * 100)
    print(util.accuracy(sentiment_classifier, test) * 100)
    return sentiment_classifier

In [25]:
def build_corpus(index, dirname, dictionary, nodelimit = None):
    """
    function takes as inputs:
        index(str) as optioned by Amazon
        dirname(str) as chosen by user for folder for output
        dictionary(dic) of pair matched parent node IDs, with keys as 'f' or 'm'
        nodelimit(list) as a range of lower and upper limit to the child nodes
    function retrieves item descriptions by calling save_item_files() for the node itself,
        and child nodes up to the numerical nodelimit
    function outputs:
        Corpus, an NLTK categorized corpus, labeling each item item
    """
    corpusdir = dirname + "/"
    if not os.path.isdir(corpusdir):
        os.mkdir(corpusdir)
    
    for k,v in dictionary.items():
        save_item_files(corpusdir, index, v , tag = k)
        for i, childnode in enumerate(get_child_nodes(v)):
            if nodelimit is None:
                save_item_files(corpusdir, index, childnode, tag = k)
            elif nodelimit != None i >= nodelimit[0] and i <= nodelimit[1]:
                save_item_files(corpusdir, index, childnode, tag = k)
            else:
                continue
            
    Corpus = CategorizedPlaintextCorpusReader(corpusdir, r'.*.txt', cat_pattern=r'\w+_([fm]).txt')
    return Corpus

SyntaxError: invalid syntax (<ipython-input-25-a0596086a4c1>, line 22)

In [10]:
def get_FD(corpus, cat = None):
    # Function takes categorized NLTK corpus and returns FreqDist object while printing 15 most common words of corpus
    words = corpus.words(categories = cat)
    filtered = filter_lower(words)
    FD = FreqDist(filtered)
    print(FD.most_common(15))
    return FD

In [11]:
def create_features(corpus, cats = ['f','m']):
    # function takes categorized NLTK corpus, and creates a dictionary of features labeled 'f' or 'm'
    dictout = {}
    for cat in cats:
        dictout[cat] = [
        (bag_filter_lower(corpus.words(fileids=[f])), cat) \
        for f in corpus.fileids(categories=cat)]
    return dictout

In [12]:
def save_item_files(corpusdir, SearchIndex, Node, tag=""):
    """
    function takes as inputs:
        corpusdir(str) of the directory where corpus files will be created and saved to
        SearchIndex(str) of item to be searched as optioned by Amazon
        node(int) of the node ID for the item search
        tag(str) of the "tag" that goes at end of filename to indicate "f" or "m"
    function uses created Amazon api object and saves description of each item as a text file
        with item ASIN as filename, with usually the indicated tag at the end following underscore
    function returns nothing
    """
    items = api.item_search(
        SearchIndex, ResponseGroup = 'EditorialReview', BrowseNode = Node)

    for i, item in enumerate(items):
        try:
            content = item.EditorialReviews.EditorialReview.Content.text
            content = re.sub('<[^<]+?>', '', content)
            with open(corpusdir + item.ASIN.text + "_" + tag + '.txt','w') as fout:
                fout.write(content.encode('utf-8'))
                
        except AttributeError:
            pass

In [13]:
def filter_lower(words):
    # function filters for stopwords and punctuations from a list of words and returns list with all lowercase words
    uselesswords = stopwords.words("english") + list(string.punctuation)
    #otherwords = ["\s", "br", "b", "</", "><", ".</", "/>", ".<", ">-", """+)""", """)<\\"""]
    #uselesswords += otherwords
    return [word.lower() for word in words if not word.lower() in uselesswords]

In [14]:
def bag_filter_lower(words):
    # function filters for stopwords and punctuations from a list of words 
    # and returns dictionary with all lowercase words with value 1
    uselesswords = stopwords.words("english") + list(string.punctuation)
    return {word.lower():1 for word in words if not word.lower() in uselesswords}

In [15]:
def get_child_nodes(node_id):
    # function takes the node ID(int) and returns a list of child_nodes
    child_nodes = []
    result = api.browse_node_lookup(node_id)
    try:
        for child in result.BrowseNodes.BrowseNode.Children.BrowseNode:
            nodeout = child.BrowseNodeId.text
            child_nodes.append(nodeout)
    except AttributeError:
        print("No child nodes found")
    return child_nodes

In [16]:
def print_child_nodes(node_id):
    # function takes as input the node ID(int) and prints descriptions and node ID of children nodes
    result = api.browse_node_lookup(node_id)
    for child in result.BrowseNodes.BrowseNode.Children.BrowseNode:
         print('{} ({})'.format(child.Name, child.BrowseNodeId))

In [94]:
print_child_nodes(13271079011)

After Shaves (13271113011)
Beard & Mustache Care (13271118011)
Body Groomers (7697066011)
Depilatories (3779081)
Electric Shavers (13271092011)
Eyebrow Trimmers (7706129011)
Nose & Ear Hair Trimmers (234950011)
Razors & Blades (13271080011)
Shaving & Grooming Sets (13271122011)
Shaving Accessories (13271115011)
Shaving Creams, Lotions & Gels (13271103011)
Tweezers (3779181)
Waxing (3779191)


In [93]:
print_child_nodes(13269990011)

Bleaching (3779061)
Depilatories (3779081)
Electric Shavers (13295180011)
Epilators, Groomers & Trimmers (13269998011)
Laser, Light & Electrolysis Hair Removal (13406780011)
Razors & Blades (13269991011)
Shaving & Grooming Sets (13270009011)
Shaving Creams, Lotions & Gels (13270001011)
Tweezers (3779181)
Waxing (3779191)


In [95]:
print_child_nodes(razor['f'])

Cartridges & Refills (13269995011)
Disposable Razors (13269997011)
Razor Systems (13269993011)
Razors with Soap Bars (13295188011)


In [96]:
print_child_nodes(razor['m'])

Razor Systems (13271082011)
Cartridges & Refills (13271084011)
Disposable Razors (13271086011)
Safety Razors (13271088011)
Straight Razors (13271090011)


In [25]:
print_child_nodes(socks['m'])

Casual Socks (2476509011)
Dress & Trouser Socks (1045726)
Athletic Socks (1045724)


In [24]:
print_child_nodes(socks['f'])

Casual Socks (2376196011)
Dress & Trouser Socks (2376197011)
Athletic Socks (1044920)
Leg Warmers (2376198011)
Tights (1044936)
Sheers (1044934)
Slipper Socks (2376199011)
No Show & Liner Socks (2376200011)


In [97]:
print_child_nodes(shirts['f'])

Compression Tops (9590778011)
Polo Shirts (11444121011)
T-Shirts (11444122011)
Tank Tops (11444123011)


In [98]:
print_child_nodes(shirts['m'])

Compression Tops (9590779011)
Polo Shirts (11444074011)
T-Shirts (11444075011)
Tank Tops (11444076011)


In [17]:
print_child_nodes(perfume['f'])

Body Sprays (3783161)
Cologne (11057051)
Eau Fraiche (16262036011)
Eau de Parfum (11057071)
Eau de Toilette (11057081)
Essential Oils (11057091)
Sets (11057111)


In [18]:
print_child_nodes(perfume['m'])

Body Sprays (16262034011)
Cologne (11059721)
Eau Fraiche (16262035011)
Eau de Parfum (363235011)
Eau de Toilette (363236011)
Sets (11059741)


In [112]:
#print_child_nodes(3764441)
print_child_nodes(3760931)

Baby & Child Care (10787321)
Health Care (3760941)
Household Supplies (15342811)
Medical Supplies & Equipment (3775161)
Oral Care (10079992011)
Personal Care (3777891)
Sexual Wellness (3777371)
Sports Nutrition (6973663011)
Stationery & Gift Wrapping Supplies (723418011)
Vision Care (10079994011)
Vitamins & Dietary Supplements (3764441)
Wellness & Relaxation (10079996011)


In [33]:
print_child_nodes(11443885011)

Insulated Pants (11443887011)
Shell Pants (11443888011)


In [34]:
print_child_nodes(11443933011)

Capri Pants (11443934011)
Insulated Pants (11443935011)
Shell Pants (11443936011)


In [76]:
print_child_nodes(shoes_running['f'])

Road Running (14210388011)
Track & Field & Cross Country (3412255011)
Trail Running (1264582011)


In [77]:
print_child_nodes(shoes_running['m'])

Road Running (14210389011)
Track & Field & Cross Country (3420973011)
Trail Running (1264575011)
